本项目利用无监督学习进行股票数据分析。     
假设我们并不知道股票市场有多少集群，因此需要用一种近邻传播聚类算法来集群。     
这种算法会找出数据中每个集群的代表性数据点，会找到数据点之间的相似性度量，并把所有的数据点看成潜在的
代表性数据点，也称为取样器

In [16]:
import json
import datetime

import numpy as np
import matplotlib.pyplot as plt
from sklearn import covariance, cluster
from mpl_finance import uotes_historical_yahoo_ochl as quotes_yahoo

# 输入符号信息文件
symbol_file = 'symbol_map.json'
# 加载符号映射信息
with open(symbol_file, 'r') as f:
    symbol_dict = json.loads(f.read())
symbols, names = np.array(list(symbol_dict.items())).T

# 指定分析时间段
start_date = datetime.datetime(2004, 4, 5)
end_date = datetime.datetime(2007, 6, 2)

# 读取输入数据
quotes = [quotes_yahoo(symbol, start_date, end_date, asobject=True) 
                for symbol in symbols]

# 使用每天的开盘价和收盘价的差异来分析数据
# 提取开盘价和收盘价
opening_quotes = np.array([quote.open for quote in quotes]).astype(np.float)
closing_quotes = np.array([quote.close for quote in quotes]).astype(np.float)
# 计算每日股价波动
delta_quotes = closing_quotes - opening_quotes

# 建立一个协方差图模型
edge_model = covariance.GraphLassoCV()
# 数据标准化
X = delta_quotes.copy().T
X /= X.std(axis=0)
# 训练模型
with np.errstate(invalid='ignore'): # 用来忽略警告
    edge_model.fit(X)
# 用近邻传播算法建立聚类模型
_, labels = cluster.affinity_propagation(edge_model.covariance_)
num_labels = labels.max()
# 打印聚类结果
for i in range(num_labels + 1):
    print("Cluster", i+1, "-->", ', '.join(names[labels == i]))


ImportError: cannot import name 'uotes_historical_yahoo_ochl'